In [2]:
from __future__ import division
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from  operator import add
import datetime
from operator import itemgetter

In [333]:
# Functions for extracting:
def parse(row):
    serialNumber = str(row.value.serialNumber)
    timestamp = row.time
    if row.value.zone is not None:
        zone = str(row.value.zone.value.name)
    else:
        zone = None
    if row.value.sGroup is not None:
        group = str(row.value.sGroup.value)
    else:
        group = None  # -> No group???
    if row.value.status is not None:
        if str(row.value.status.value) == 'Sold':
            status = int(1)
        else:
            status = int(0)
    else:
        status = int(0)
        # if row.value.itemcode is not None:
        #   itemcode=str(row.itemcode.value)
        # else:
        #   itemcode=None
    return [serialNumber, timestamp, zone, group, status]


def summarize_serial(x, y):
    # Assign to first_date the first blink row:
    last_date = max(x[0], y[0])
    if last_date == x[0]:
        first_line = y
        last_line = x
    elif last_date == y[0]:
        first_line = x
        last_line = y
    timeSpent=x-y
    
    
    status = last_line[3]
    zone = first_line[1]
    group = last_line[2]
    # brand = last_line[4]
    # dep = last_line[5]
    # blinks_number = first_line[6] + last_line[6]
    # result = [first_date, last_line[0], zone, group, brand, dep, blinks_number, status]
    result = [last_date, zone, group, status]
    return result


def extracting_time(x):
    year = x[0].year
    month = x[0].month
    WM = (x[0].day - 1) // 7 + 1
    DM = x[0].day
    DW = x[0].weekday()
    Zone = x[1]
    Sold = x[2]
    return [year, month, WM, DM, DW, Zone, Sold]

# Falta hacer la suma: 
def get_delta_time(keyValue):
    # The list of blinks is in keyValue[1]:
    list_blinks=keyValue[1]
    # Sorting by time:
    list_blinks.sort(key=itemgetter(0))
    # Create an example_res:
    example_res=range(0,len(list_blinks))
    # Getting the list of blinks with dwelling time
    for i in range(0,len(list_blinks)):
        if i<len(list_blinks)-1:
            example_res[i]=list(list_blinks[i])+[list_blinks[i+1][0]-list_blinks[i][0]]
        else:
            example_res[i]=list(list_blinks[i])+[datetime.timedelta(0)]
    # Ordering it:        
    example_res.sort(key=itemgetter(4))
    # Chosing the one with 
    return (keyValue[0],example_res[-1])

In [10]:
sqlContext.sql(
    "CREATE TEMPORARY TABLE thingsTable USING com.stratio.datasource.mongodb OPTIONS (host 'localhost:27017', "
    "database 'riot_main', collection 'thingSnapshotsSH', splitKey 'time', splitKeyType 'isoDate', "
    "splitKeyMin '2016-01-01T00:00:00.000Z', splitKeyMax '2016-01-05T00:00:00.000Z' )")

# Extract all blinks from the Database:
raw_data = sqlContext.sql(
    "SELECT * FROM thingsTable WHERE value.groupId=3 AND value.status.value='Sold'")  # WHERE value.status.value = 'Sold'") --AND value.status.value='Sold'
print ('Number of blinks: ' + str(raw_data.count()))
#print ('Document Example:    ' + str(raw_data.first()))

Number of blinks: 13173


In [9]:
#test_data.coalesce(1, True).saveAsPickleFile('/tmp/BlinksZone3M')

In [343]:
# Getting the test data:
test_data= raw_data.map(parse).map(lambda x: (x[0], (x[1], x[2], x[3], x[4])))
print(test_data.first())

('AE1000000000000000489989', (datetime.datetime(2016, 1, 1, 0, 0, 41, 643000), 'IT', None, 1))


In [334]:
# Getting the zone by dwell-time:
grouped=test_data.groupByKey().map(lambda x : (x[0], list(x[1])))   # Cambiar Group by Key por ReduceByKey
groupedDwellTime=grouped.map(get_delta_time)

In [327]:
# Getting the rows and extract as (serial, initial_date, end_date, zone, group, status(binary))
SerialDateZoneGroupStatusRdd=groupedDwellTime.map(lambda x:(x[0],x[1][0].date(),x[1][1],x[1][2],x[1][3]))
print ('Number of Things Sold: ' + str(SerialDateZoneGroupStatusRdd.count()))
print ('Thing Sold Example :    ' + str(SerialDateZoneGroupStatusRdd.first()))

Number of Things Sold: 5411
Thing Sold Example :    ('AE1000000000000000574319', datetime.date(2016, 1, 2), 'Stock Room 1', None, 1)


In [341]:
# Get the number of objects sold per Zone per day:
DateZoneStatusRdd = SerialDateZoneGroupStatusRdd.map(
    lambda x: ((x[1], x[2]), x[-1])).reduceByKey(add).map(lambda x: (x[0][0], x[0][1], x[1]))

print ('Number of Zones Sold :' + str(DateZoneStatusRdd.count()))
print ('Zone Example :    ' + str(DateZoneStatusRdd.first()))

Number of Zones Sold :76
Zone Example :    (datetime.date(2016, 1, 2), 'IT Accessories', 48)


In [342]:
# Extracting Features to RDDs:
ZoneData = DateZoneStatusRdd.map(extracting_time)
print ('Number of registers : ' + str(ZoneData.count()))
print ('ZoneData Example :    ' + str(ZoneData.first()))

Number of registers : 76
ZoneData Example :    [2016, 1, 1, 2, 5, 'IT Accessories', 48]


## Sold per Zone per Group: NO ES NECESARIO

In [9]:
# Get the number of objects sold per Zone per Group per day:
DateZoneGroupStatusRdd = SerialDateZoneGroupStatusRdd.reduceByKey(summarize_serial).map(
    lambda x: ((x[1][0].date(), x[1][1], x[1][2]), x[1][3])).reduceByKey(add).map(
    lambda x: (x[0][0], x[0][1], x[0][2],
               x[1]))
print ('Number of things Sold per group per zone: ' + str(DateZoneGroupStatusRdd.take(5)))
print ('Zone Group Example :    ' + str(DateZoneGroupStatusRdd.first()))

Number of things Sold per group per zone: [(datetime.date(2016, 1, 1), 'HVS', 'IT Acc Promotion', 1), (datetime.date(2016, 1, 1), 'Photography', 'Momentos', 2), (datetime.date(2016, 1, 1), 'IT Accessories', 'Networking', 4), (datetime.date(2016, 1, 1), 'Photography', 'Charger', 1), (datetime.date(2016, 1, 1), 'Gaming', 'PDA and Handheld Accessories', 5)]
Zone Group Example :    (datetime.date(2016, 1, 1), 'HVS', 'IT Acc Promotion', 1)


In [12]:
ZoneData.coalesce(1, True).saveAsPickleFile('/tmp/')

In [14]:
ZoneData.coalesce(1, True).saveAsTextFile('/tmp/ZoneObjSoldTEXt2D_v2')